<a href="https://colab.research.google.com/github/ML-Bioinfo-CEITEC/cDNA-pretraining/blob/main/experiments/genomic_benchmarks/Metrics_on_genomic_benchmarks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install -qq transformers genomic-benchmarks datasets

In [ ]:
### Parameters
MODEL_NAME = "simecek/DNADeberta2"
TOKENIZER_NAME = "armheb/DNA_bert_6"
K = 6
STRIDE = K

# if less than 1, only this fraction of each dataset is used
DATASET_THINING = 0.01

BENCHMARKS_FOLDER = '/root/.genomic_benchmarks'
# BENCHMARKS_FOLDER = '/home/jovyan/.genomic_benchmarks/' (for INFRA HUB)

DATASETS = [('demo_coding_vs_intergenomic_seqs', 0),
 ('demo_human_or_worm', 0), ('human_enhancers_cohn', 0), ('human_enhancers_ensembl', 0),
 ('human_ensembl_regulatory', 0), ('human_nontata_promoters', 0), ('human_ocr_ensembl', 0)]

BATCH_SIZE = 32
LEARNING_RATE = 8e-5
EPOCHS = 4
RUNS = 1

# do not forget to attach drive
OUTPUT_PATH = 'drive/MyDrive/genomic_benchmarks/experiment1.csv'

## Download benchmark datasets and tokenizer

In [ ]:
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check.info import is_downloaded
from pathlib import Path
from tqdm.autonotebook import tqdm

for dataset_name, dataset_version in tqdm(DATASETS):
    if not is_downloaded(dataset_name):
        download_dataset(dataset_name, version=dataset_version)

benchmark_root = Path(BENCHMARKS_FOLDER)

  0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME)

Downloading:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def kmers_strideK(s, k=K):
    return [s[i:i + k] for i in range(0, len(s), k) if i + k <= len(s)]

def kmers_stride1(s, k=K):
    return [s[i:i + k] for i in range(0, len(s)-k+1)]

if (STRIDE == 1):
  kmers = kmers_stride1
else:
  kmers = kmers_strideK

# function used for the actual tokenization
def tok_func(x): return tokenizer(" ".join(kmers(x["seq"])))

# example
tok_func({'seq': 'ATGGAAAGAGGCACCATTCT'})    

{'input_ids': [2, 501, 835, 650, 3], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

## Looping through datasets, fine-tuning the model for each of them, logging metrics

In [ ]:
import pandas as pd
import numpy as np
from random import random
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset, DatasetDict, load_metric

def compute_metrics_binary(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def compute_metrics_multi(eval_preds):
    metric = load_metric("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

outputs = []

for dataset_name, dataset_version in tqdm(DATASETS):
    

    labels = sorted([x.stem for x in (benchmark_root / dataset_name / 'train').iterdir()])

    tmp_dict = {}

    for split in ['train', 'test']:
        for nlabel, label in enumerate(labels):
            for f in (benchmark_root / dataset_name / split / label).glob('*.txt'):
                txt = f.read_text()
                if not DATASET_THINING or DATASET_THINING==1:
                    tmp_dict[f"{label} {f.stem}"] = (split, nlabel, txt)
                elif random() < DATASET_THINING:
                    tmp_dict[f"{label} {f.stem}"] = (split, nlabel, txt)

    df = pd.DataFrame.from_dict(tmp_dict).T.rename(columns = {0: "dset", 1: "cat", 2: "seq"})

    ds = Dataset.from_pandas(df)

    tok_ds = ds.map(tok_func, batched=False, remove_columns=['__index_level_0__', 'seq'])
    tok_ds = tok_ds.rename_columns({'cat':'labels'})

    dds = DatasetDict({
        'train': tok_ds.filter(lambda x: x["dset"] == "train").remove_columns('dset'),
        'test':  tok_ds.filter(lambda x: x["dset"] == "test").remove_columns('dset')
    })

    compute_metrics = compute_metrics_binary if len(labels) == 2 else compute_metrics_multi

    for _ in range(RUNS):

        model_cls = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(labels))

        args = TrainingArguments('outputs', learning_rate=LEARNING_RATE, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
            evaluation_strategy="epoch", per_device_train_batch_size=BATCH_SIZE, per_device_eval_batch_size=BATCH_SIZE*2,
            num_train_epochs=EPOCHS, weight_decay=0.01, save_steps=100000, report_to='none')
        
        trainer = Trainer(model_cls, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                          tokenizer=tokenizer, compute_metrics=compute_metrics)
        trainer.train()
        
        max_accuracy = max([x['eval_accuracy'] for x in trainer.state.log_history if 'eval_accuracy' in x])
        max_f1 = max([x['eval_f1'] for x in trainer.state.log_history if 'eval_f1' in x]) if len(labels) == 2 else np.nan
        train_runtime = max([x['train_runtime'] for x in trainer.state.log_history if 'train_runtime' in x])
        
        outputs.append((dataset_name, max_accuracy, max_f1, train_runtime))




  0%|          | 0/7 [00:00<?, ?it/s]

loading configuration file https://huggingface.co/simecek/DNADeberta2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/eaa84554db67bd0bd1237534157006a6e5a26745104c342ca1a8117c146aa517.c7f87ccb839059561ef6996c9f8559644ce254107c992b74bd0b46fbe8c80d51
Model config DebertaConfig {
  "_name_or_path": "simecek/DNADeberta2",
  "architectures": [
    "DebertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": null,
  "position_biased_input": true,
  "relative_attention": false,
  "torch_dtype": "float32",
  "transformers_version": "4.20.1

  0%|          | 0/1005 [00:00<?, ?ex/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

...

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

PyTorch: setting up devices
Using cuda_amp half precision backend
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1374
  Num Epochs = 4
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 172


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.618812,0.639769,0.671916
2,No log,0.687490,0.651297,0.693671
3,No log,0.929528,0.654179,0.636364
4,No log,0.899754,0.668588,0.676056


***** Running Evaluation *****
  Num examples = 347
  Batch size = 64
***** Running Evaluation *****
  Num examples = 347
  Batch size = 64
***** Running Evaluation *****
  Num examples = 347
  Batch size = 64
***** Running Evaluation *****
  Num examples = 347
  Batch size = 64


Training completed. Do not forget to share your model on huggingface.co/models =)




## Outputs

In [ ]:
outputs_df = pd.DataFrame(outputs, columns = ['dataset', 'accuracy', 'f1', 'train_runtime'])
outputs_df

,dataset,accuracy,f1,train_runtime
0,demo_coding_vs_intergenomic_seqs,0.877953,0.883019,12.9503
1,demo_human_or_worm,0.913386,0.915385,11.8537
2,human_enhancers_cohn,0.750000,0.695652,8.0578
3,human_enhancers_ensembl,0.775974,0.775244,15.9134
4,human_ensembl_regulatory,0.855519,NaN,25.9042
5,human_nontata_promoters,0.835052,0.849057,8.4057
6,human_ocr_ensembl,0.668588,0.693671,17.4784


In [ ]:
outputs_df.groupby('dataset').agg({'accuracy' : ['mean', 'sem'], 'f1' : ['mean','sem'], 'train_runtime': ['mean', 'sem']})

accuracy            f1     train_runtime    
                                      mean sem      mean sem          mean sem
dataset                                                                       
demo_coding_vs_intergenomic_seqs  0.877953 NaN  0.883019 NaN       12.9503 NaN
demo_human_or_worm                0.913386 NaN  0.915385 NaN       11.8537 NaN
human_enhancers_cohn              0.750000 NaN  0.695652 NaN        8.0578 NaN
human_enhancers_ensembl           0.775974 NaN  0.775244 NaN       15.9134 NaN
human_ensembl_regulatory          0.855519 NaN       NaN NaN       25.9042 NaN
human_nontata_promoters           0.835052 NaN  0.849057 NaN        8.4057 NaN
human_ocr_ensembl                 0.668588 NaN  0.693671 NaN       17.4784 NaN

In [ ]:
# saving outputs to csv file
outputs_df.to_csv(OUTPUT_PATH, index=False)